<a id = "1.0"></a>
<p style="font-size:15px;color:#34656d;font-family:'Comic Sans MS';text-align:right;border-radius:5px 5px;">
    <i>Aprender una técnica no es un fin en si mismo, simplemente indica por donde hay que empezar</i><br><b>Proverbio Japonés</b></p> 

<a id = "1.0"></a>
<p style="font-size:15px;color:#34656d;font-family:'Comic Sans MS';text-align:right;border-radius:5px 5px;">
<i>Material curado y organizado por el Prof. Dr. Hernán D. Merlino</i><br> 
<i>NLP Versión 1.0 - 2022 Q2</i></p> 

<a id = "1.0"></a>
<p style="font-size:25px;color:#34656d;font-family:'Comic Sans MS';text-align:center;border-radius:5px;">
<strong>Auto NLP - Wikipedia Article Analizer</strong></p>
<p style="font-size:25px;color:#34656d;font-family:'Comic Sans MS';text-align:center;border-radius:5px 5px;">
<strong>_____________________________________________________________</strong></p> 

# Parametrización del análisis

In [1]:
# Numero de registros usados en el sampling del Dataset
SAMPLE_SIZE = 1000000

In [2]:
# Cantidad de tópicos a agrupar en el modelo NPL
TOPICS_NUMBER = 7

In [3]:
# Palabras a excluir del análisis
STOPWORDS = ['style', 'text', 'align', 'center', 'right', 'background', 
             'leave', 'fff', 'width', 'px', 'text', 'vertical', 'wikitable',
             'category', 'also', 'know', 'may', 'reference', 'read', 'call', 'find'
             'include', 'file', 'date', 'last', 'note', 'image', 'left', 'bgcolor', 'color',
             'font', 'rowspan', 'solid', 'value', 'access']

# Import de dependencias

In [4]:
import os
from os import path
import glob

In [5]:
import spacy
spacy.load('en_core_web_sm')

In [6]:
from matplotlib import pyplot as plt

In [7]:
import warnings
warnings.filterwarnings("ignore")

In [8]:
import pycaret
from pycaret.nlp import *

In [9]:
from pycaret.datasets import get_data

In [10]:
import pandas

# Carga de Dataset y sampleo

In [11]:
%%time
data_raw = pandas.concat(map(pandas.read_json, glob.glob(os.path.join('', "./dataset/*.json"))))

CPU times: user 1.03 s, sys: 573 ms, total: 1.6 s
Wall time: 1.63 s


In [12]:
data_raw.shape

(66691, 3)

In [13]:
data_raw.head()

,id,text,title
0,11524759,"Albert William Cretella (April 22, 1897 – May ...",Albert W. Cretella
1,11524760,Alexander B. Montgomery Alexander Brooks Montg...,Alexander B. Montgomery
2,11524761,The Roman Catholic Archdiocese of Gagnoa () is...,Roman Catholic Archdiocese of Gagnoa
3,11524762,"Alexander Evans (September 13, 1818 – December...",Alexander Evans (American politician)
4,11524764,"Alexander McKim (January 10, 1748January 18, 1...",Alexander McKim


In [14]:
data = data_raw.sample(SAMPLE_SIZE if SAMPLE_SIZE < data_raw.shape[0] else data_raw.shape[0], random_state=786).reset_index(drop=True)

In [15]:
data.shape

(66691, 3)

In [16]:
data.head()

,id,text,title
0,11587688,George A. Bicknell. George Augustus Bicknell (...,George A. Bicknell
1,58425844,Alejandra Guerrero Rodríguez (b. 5 November 19...,Alejandra Guerrero Rodríguez
2,60552607,"Thomas A. Bolger (April 1, 1887-April 16, 1953...",Thomas A. Bolger
3,52581999,"Petrolisthes novaezelandiae, known as the red ...",Petrolisthes novaezelandiae
4,11590541,"Charles Franklin Mitchell (February 18, 1806 -...",Charles F. Mitchell


# Setup de pycaret

La siguiente ejecución realiza el pre procesamiento del texto:

1. Remover números: All numeric characters are removed from the text. They are replaced with blanks.

2. Remover carateres especiales: All non-alphanumeric special characters are removed from the text. They are also replaced with blanks.

3. Tokenizar palabras: Word Tokenization: Word tokenization is the process of splitting a large sample of text into words. This is the core requirement in natural language processing tasks where each word needs to be captured separately for further analysis.

4. Remover stopwords: A stop word (or stopword) is a word that is often removed from text because it is common and provides little value for information retrieval, even though it might be linguistically meaningful. Example of such words in english language are: "the", "a", "an", "in" etc.

5. Extracción de Bigramas: A bigram is a sequence of two adjacent elements from a string of tokens, which are typically letters, syllables, or words. For example: word New York is captured as two different words "New" and "York" when tokenization is performed but if it is repeated enough times, Bigram Extraction will represent the word as one i.e. "New_York".

6. Extracción de Trigramas: Similar to bigram extraction, trigram is a sequence of three adjacent elements from a string of tokens.

7. Lemmatization: Lemmatization identifies the inflected forms of a word and returns its base form (e.g. “better” is lemmatized as “good”.

8. Custom Stopwords: Many times text contains words that are not stopwords by the rule of language but they add no or very little information. For example, in this tutorial we are using the loan dataset. As such, words like "loan", "bank", "money", "business" are too obvious and adds no value.

In [17]:
%%time
# Initialize the training environment and create transformation pipeline
exp_nlp101 = setup(data = data, target = 'text', session_id = 123, verbose=True, custom_stopwords=STOPWORDS)

Description,Value
session_id,123
Documents,66691
Vocab Size,214129
Custom Stopwords,True


CPU times: user 15min 37s, sys: 42.8 s, total: 16min 19s
Wall time: 17min 1s


In [18]:
models()

,Name,Reference
ID,,
lda,Latent Dirichlet Allocation,gensim/models/ldamodel
lsi,Latent Semantic Indexing,gensim/models/lsimodel
hdp,Hierarchical Dirichlet Process,gensim/models/hdpmodel
rp,Random Projections,gensim/models/rpmodel
nmf,Non-Negative Matrix Factorization,sklearn.decomposition.NMF


# LDA: Entrenamiento por tópicos

In [19]:
%%time
lda = create_model(model='lda', multi_core=True, num_topics=TOPICS_NUMBER)

CPU times: user 2min 48s, sys: 1min 57s, total: 4min 46s
Wall time: 4min 54s


In [20]:
print(lda)

LdaMulticore<num_terms=214129, num_topics=7, decay=0.5, chunksize=100>


In [21]:
%%time
# [OPTATIVO]: Optimización del modelo
#lda = tune_model(model='lda', estimator='knn', optimize='Accuracy')

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.01 µs


In [22]:
# Guardar el modelo
save_model(lda, 'wikipedia_araticle_model_lda_sample_{}_topics_{}'.format(SAMPLE_SIZE, TOPICS_NUMBER))

Model Succesfully Saved


(<gensim.models.ldamulticore.LdaMulticore at 0x7ff255e977f0>,
 'wikipedia_araticle_model_lda_sample_1000000_topics_7.pkl')

# Análisis de Resultados

In [23]:
%%time
lda_results = assign_model(lda)

CPU times: user 45.1 s, sys: 4.29 s, total: 49.4 s
Wall time: 45.4 s


In [24]:
lda_results.head(60)

,id,text,title,Topic_0,Topic_1,Topic_2,Topic_3,Topic_4,Topic_5,Topic_6,Dominant_Topic,Perc_Dominant_Topic
0,11587688,bear complete city move least son daughter ele...,George A. Bicknell,0.003612,0.810828,0.003610,0.171104,0.003625,0.003610,0.003611,Topic 1,0.81
1,58425844,master recognize woman master career chess cha...,Alejandra Guerrero Rodríguez,0.002854,0.287961,0.002857,0.002858,0.002860,0.002853,0.697757,Topic 6,0.70
2,60552607,bear farm live serve education serve bolger se...,Thomas A. Bolger,0.008437,0.568189,0.008434,0.389652,0.008443,0.008420,0.008427,Topic 1,0.57
3,52581999,petrolisthe red half crab red false crab speci...,Petrolisthes novaezelandiae,0.002833,0.002843,0.930962,0.002832,0.054872,0.002830,0.002829,Topic 2,0.93
4,11590541,fifth twenty_sixth congress bear burial record...,Charles F. Mitchell,0.188228,0.486331,0.030510,0.277773,0.015606,0.000774,0.000779,Topic 1,0.49
5,52417909,bear make first class debut saurashtra make li...,Jay Chauhan (cricketer),0.008490,0.008494,0.008482,0.008501,0.008487,0.008481,0.949066,Topic 6,0.95
6,60436809,european come sometime fight pension applicati...,Martin Brechall,0.001405,0.196784,0.001401,0.324147,0.388360,0.086496,0.001408,Topic 4,0.39
7,7762817,originally renga studio creator partner includ...,Renga Media,0.599876,0.012739,0.000393,0.210973,0.000393,0.175232,0.000393,Topic 0,0.60
8,52488519,man tour compose sixth annual pre open era tou...,1882 Men's Tennis tour,0.000534,0.000533,0.000535,0.079767,0.141020,0.000533,0.777077,Topic 6,0.78
9,52418740,bear football player bear football player,Kaikaew,0.020408,0.020431,0.020409,0.020432,0.020409,0.020408,0.877502,Topic 6,0.88


In [40]:
lda_results[lda_results.Dominant_Topic == "Topic 6"].head(30)

,id,text,title,Topic_0,Topic_1,Topic_2,Topic_3,Topic_4,Topic_5,Topic_6,Dominant_Topic,Perc_Dominant_Topic
1,58425844,master recognize woman master career chess cha...,Alejandra Guerrero Rodríguez,0.002854,0.287961,0.002857,0.002858,0.002860,0.002853,0.697757,Topic 6,0.70
5,52417909,bear make first class debut saurashtra make li...,Jay Chauhan (cricketer),0.008490,0.008494,0.008482,0.008501,0.008487,0.008481,0.949066,Topic 6,0.95
8,52488519,man tour compose sixth annual pre open era tou...,1882 Men's Tennis tour,0.000534,0.000533,0.000535,0.079767,0.141020,0.000533,0.777077,Topic 6,0.78
9,52418740,bear football player bear football player,Kaikaew,0.020408,0.020431,0.020409,0.020432,0.020409,0.020408,0.877502,Topic 6,0.88
15,60526319,tournament play clay_court third edition tourn...,2019 Internazionali di Tennis d'Abruzzo,0.003291,0.003291,0.003290,0.003293,0.003296,0.003304,0.980233,Topic 6,0.98
18,60556828,perm_krai_russia_population,Baskiye,0.071454,0.071454,0.071454,0.071454,0.071454,0.071454,0.571278,Topic 6,0.57
19,58443220,exhibition benefit shriner child play televise...,2019 East–West Shrine Game,0.000973,0.111305,0.000974,0.081817,0.000975,0.000974,0.802982,Topic 6,0.80
25,60608191,play represent national football team match it...,Antonio Fayenz,0.014289,0.014303,0.014289,0.014303,0.014297,0.014290,0.914228,Topic 6,0.91
30,60602042,bear mexican bear colombian actor bear footbal...,Alejandro Martínez,0.015947,0.015928,0.015897,0.265680,0.015904,0.015894,0.654750,Topic 6,0.65
40,58470366,list transaction take place season turkish wom...,List of 2018–19 KBSL season transactions,0.033170,0.100868,0.007271,0.007514,0.000405,0.000405,0.850366,Topic 6,0.85


### Frequency

In [25]:
plot_model(model=lda, plot = 'frequency', topic_num= "Topic 0")

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceed

In [26]:
plot_model(model=lda, plot = 'frequency', topic_num= "Topic 1")

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceed

In [27]:
plot_model(model=lda, plot = 'frequency', topic_num= "Topic 2")

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceed

In [28]:
plot_model(model=lda, plot = 'frequency', topic_num= "Topic 3")

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceed

In [29]:
plot_model(model=lda, plot = 'frequency', topic_num= "Topic 4")

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceed

### Bigrams

In [30]:
plot_model(model=lda, plot = 'bigram', topic_num= "Topic 0")

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceed

In [31]:
plot_model(model=lda, plot = 'bigram', topic_num= "Topic 1")

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceed

In [32]:
plot_model(model=lda, plot = 'bigram', topic_num= "Topic 2")

AssertionError: 

In [ ]:
plot_model(model=lda, plot = 'bigram', topic_num= "Topic 3")

In [ ]:
plot_model(model=lda, plot = 'bigram', topic_num= "Topic 4")

### Trigram

In [ ]:
plot_model(model=lda, plot = 'trigram', topic_num= "Topic 0")

In [ ]:
plot_model(model=lda, plot = 'trigram', topic_num= "Topic 1")

In [ ]:
plot_model(model=lda, plot = 'trigram', topic_num= "Topic 2")

In [ ]:
plot_model(model=lda, plot = 'trigram', topic_num= "Topic 3")

In [ ]:
plot_model(model=lda, plot = 'trigram', topic_num= "Topic 4")

### End